## Подготовка к контрольной по эконометрике

Основные темы, которые необходимо знать на контрольной:

1. Тест Чоу
2. Гетероскедастичность
    - тест Голдфильда-Квандта
    - тест Уайта
3. Автокорелляция
    - тест Дарбина-Уотсона
4. ошибки спецификации
    - тест Бокса-Кокса
    - тест Рамсея
    
### Тест Чоу

#### Для чего применяется?

Тест Чоу позволяет оценить значимость улучшения регрессионной модели после разделения исходной выборки на части.

Пусть на временном интервале $[T_1;T_3]$ прогноз для момента t по уже полученным данным  $y_t$ имеет следующий вид:

$$\hat{y}_t=\sum_{j=1}^kf_j(t)\alpha_j$$

Выделим внутри рассматриваемого временного интервала момент T_2. Пусть прогноз на отрезке

$$[T_1;T_2]:\; \hat{y}_{1t}=\sum_{j=1}^{k_1}f_{1j}(t)\alpha_{1j}$$
$$[T_2;T_3]:\; \hat{y}_{2t}=\sum_{j=1}^{k_2}f_{2j}(t)\alpha_{2j}$$

С помощью теста Чоу можно понять необходимо ли менять модель в момент времени $T_2$.

#### Рецепт приготовления теста

Пусть $\epsilon_{1t}=\hat{y}_{1t}-y_t$ и $\epsilon_{2t}=\hat{y}_{2t}-y_t$ - неточности в предсказании рядов на отрезках  $[T_1;T_2]$ и $[T_2;T_3]$ соответственно.
Будем считать, что $\epsilon_{1t},\; \epsilon_{2t}$ распределены нормально с одними и теми же параметрами.

Сформулируем нулевую гипотезу:

$$H_0:\; (1) \sim (2)$$

Будем использовать следующие обозначения:

$\epsilon_t=\hat{y}_t-y_t$ - неточность в предсказании ряда на отрезке $[T_1;T_3]$

$RSS$ - остаточная сумма квадратов для всего интервала $[T_1;T_3]$

$RSS_1=\sum_{t=T_1}^{T_2-1}\epsilon_{1t}^2$

$RSS_2=\sum_{t=T_2}^{T_3}\epsilon_{2t}^2$

Статистика Чоу:

$$F(T_2)=\frac{(RSS - RSS_1 - RSS_2) / k}{(RSS_1 + RSS_2) / (n_1 + n_2 - 2k)}$$

Статистика Чоу имеет распределение Фишера с $k$ и $n_1 + n_2 - 2k$ степенями свободы.

Для критерия Чоу критическая область при уровне значимости  $\alpha$ - это область

$$\Omega_{\alpha}:\; F(T_2)>F_{k, n_1 + n_2 - 2k,\alpha}$$

где $F_{k, n_1 + n_2 - 2k,\alpha}$ - квантиль Фишера.

Если гипотеза $H_0$ отвергается, то необходимо использовать две модели.

#### Пример теста Чоу

Подготавливаем данные для нашего тестирования:

In [1]:
import numpy as np
import statsmodels.api as sm

nsample = 1000
x = np.linspace(0, 100, nsample)
X = np.column_stack((x, x**2))
beta1 = np.array([1, 0.1, 10])
e = np.random.normal(size=nsample)

X = sm.add_constant(X)
y1 = np.dot(X, beta1) + e

beta2 = np.array([1, 2, 5])
e = np.random.normal(size=nsample)

y2 = np.dot(X, beta2) + e

X = np.row_stack((X, X))
y = np.append(y1, y2)

Таким образом мы сгенерировали набор данных, сгенерированный двумя разными моделями, проверим, сможет ли тест определить необходимость использования разных моделей. Итак, построим соответствующие модели:

In [2]:
from scipy import stats

results = sm.OLS(y, X).fit()
# для первой модели берем первую половину данных
results_1 = sm.OLS(y[nsample:], X[nsample:]).fit()
# для второй модели берем вторую половину данных
results_2 = sm.OLS(y[:nsample], X[:nsample]).fit()

stats.f.ppf(0.95, 1, 1)

k = results.df_model + 1
n1 = results_1.nobs
n2 = results_2.nobs

# считаем значение тестовой статистики
RSS = results.ssr
RSS1 = results_1.ssr
RSS2 = results_2.ssr

T = ((RSS - (RSS1 + RSS2)) / k) / ((RSS1 + RSS2) / (n1 + n2 - 2 * k))

print("Значение статистики: %f" % T)
print("Критическое значение для статистики: %f" % stats.f.ppf(0.95, k, n1 + n2 + k))

Значение статистики: 83242579327.319168
Критическое значение для статистики: 2.609345


Значение статистики больше полученного критического значения, значит необходимо использовать разные модели.



### Тест Голдфильда-Квандта

#### Для чего применяется?

Тест проводится для проверки наличия условной гетероскедастичности от какого либо параметра модели.

- требуется нормальность ошибок
- тест подходит для малых выборок

#### Рецепт приготовления теста

Сформулируем нулевую гипотезу: имеет место условная гомоскедастичность

$$H_0:\; Var(\epsilon_i | X) = \sigma^2$$

1. Сортируем наблюдения по предполагаемому убыванию условной дисперсии
2. Выкидываем часть наблюдений из середины (1/4 наблюдений)
3. Оцениваем исходную модель отдельно по первым и последним наблюдениям
4. Получаем $RSS_1$ и $RSS_2$
5. Считаем тестовую статистику $$GQ = \frac{RSS_1 (n_1 - k)}{RSS_2 (n_2 - k)}$$

$n_1$ - число наблюдений в верхней части

$n_2$ - число наблюдений в нижней части

$k$ - число параметров модели

Статистика Голдфильда-Квандта (GQ) имеет распределение Фишера с $n_1 - k$ и $n_2 - k$ степенями свободы.

Для критерия $GQ$ критическая область при уровне значимости  $\alpha$ - это область

$$\Omega_{\alpha}:\; GQ > F_{n_1 - k, n_2 - k, \alpha}$$

где $F_{n_1 - k, n_2 - k, \alpha}$ - квантиль Фишера.

Если гипотеза $H_0$ отвергается, то можно говорить о наличии условной гетероскедастичности по выбранному параметру.

#### Пример теста Голдфильда-Квандта

$$TODO$$

### Тест Уайта

#### Для чего применяется?

Содержательный смысл теста в том, что часто гетероскедастичность модели вызвана зависимостью (возможно довольно сложной) дисперсий ошибок от признаков. Реализуя эту идею, Уайт предложил метод тестирования гипотезы $H_0$ (наличие условной гомоскедастичности) без каких-либо предположений о структуре гетероскедастичности.

#### Рецепт приготовления теста

Сформулируем нулевую гипотезу: имеет место условная гомоскедастичность

$$H_0:\; Var(\epsilon_i | X) = \sigma^2$$

1. Оценить основную регрессию, получить остатки $\epsilon_{i}$
2. Оценить вспомогательную регрессию: $$\epsilon_{i}^2 = \gamma_1 + \sum_{j = 2}^{m} \gamma_j z_{ij} + u_i$$
3. Посчитать статистику Уайта: $$W = nR^2$$

$R^2$ - коэффициент детерминации для вспомогательной регрессии

$z_{im}$ - факторы, определяющие форму гетероскедастичности, можно брать следующие:
- изначальные факторы основной модели
- попарные произведения факторов
- квадраты факторов

Статистика Уайта (W) имеет распределение $\chi^2$ с $m - 1$ степенью свободы.

Для критерия $W$ критическая область при уровне значимости  $\alpha$ - это область

$$\Omega_{\alpha}:\; W > \chi^2_{m - 1, \alpha}$$

где $\chi^2_{m - 1, \alpha}$ - квантиль $\chi^2$-распределения.

Если гипотеза $H_0$ отвергается, то можно говорить о наличии условной гетероскедастичности по выбранному параметру.

#### Пример теста Уайта

$$TODO$$

### Тест Дарбина-Уотсона

#### Для чего применяется?

Данный тест используется для проверки наличия автокорреляции первого порядка элементов исследуемой последовательности.

#### Рецепт приготовления теста

Сформулируем нулевую гипотезу: условная автокорелляция отстутствует

$$H_0:\; \rho = 0$$

1. Оценить регрессию, получить остатки $\epsilon_{i}$
2. Считаем статистику Дарбина-Уотсона: $$DW =  \frac{\sum_{i = 2}^T (\hat{\epsilon}_{i} - \hat{\epsilon}_{i- 1})^2}{\sum_{i = 1}^T \hat{\epsilon}_{i}^2}$$
3. Оцениваем коэффициент автокорелляции первого порядка:

$$\rho = 1 - \frac{DW}{2} $$

Качественные выводы для значения $\hat{\rho}$:

- $\hat{\rho} = 1$ - сильная положительная автокорреляция
- $\hat{\rho} = 0$ - отсутствие автокорреляции
- $\hat{\rho} = -1$ - сильная отрицательная автокорреляция

#### Пример теста Дарбина-Уотсона

$$TODO$$

### Тест Рамсея

#### Для чего применяется?

Применяется для тестирования функциональной формы (спецификации) модели.

#### Рецепт приготовления теста

1. Оцениваем исходную регрессию $Y_i = \alpha + \sum_{j = 1}^{m} \beta_j X_ij$, получаем оценки $\hat{Y}_i$
2. Строим вспомогательную регрессию: $$Y_i = \alpha + \sum_{j = 1}^{m} \beta_j X_ij + \sum_{k = 2}^{g} \gamma_k \hat{Y_{i}^{k}} $$

Сформулируем нулевую гипотезу: 

$$H_0:\; \forall k: \gamma_k = 0$$

Данную гипотезу проверяют с помощью F-теста или LR-теста.

$$T = \frac{(RSS_R - RSS_{UR}) / g}{RSS_{UR} / (n - m - g)}$$

Статистика (T) имеет распределение Фишера с $g$ и $n - m - g$ степенями свободы.

$$T \sim F(g, n - m - g)$$

Для критерия $T$ критическая область при уровне значимости  $\alpha$ - это область

$$\Omega_{\alpha}:\; T > F_{g, n - m - g, \alpha}$$

где $F_{g, n - m - g, \alpha}$ - квантиль Фишера.

#### Пример теста Рамсея

$$TODO$$



### Тест Бокса-Кокса

#### Для чего применяется?

Применяется для тестирования функциональной формы (спецификации) модели в части вида зависимой переменной, а именно отвечает на вопрос необходимо ли использовать логарифм зависимой переменной или само значение.

Таким образом выбирается лучшая модель из следующих:
$$Y_i = \alpha + \sum_{j = 1}^{m} \beta_j X_ij$$
$$ln(Y_i) = \alpha + \sum_{j = 1}^{m} \beta_j X_ij$$

#### Рецепт приготовления теста

Сформулируем нулевую гипотезу: 

$$H_0:\; (1) \sim (2)$$

1. Нормируем наблюдения $Y_i$ на их среднее геометрическое (зачем это делать можно почитать вот [тут](http://eos.ibi.spb.ru/umk/4_5/6/print/6_P1_R1_T4.pdf)): $$Y_i^* = \frac{Y_i}{(\prod_{i = 1}^{n} Y_i)^{\frac{1}{n}}}$$
2. Строим две вспомогательные регрессии: $$Y_i^* = \alpha + \sum_{j = 1}^{m} \beta_j X_ij$$ $$ln(Y_i^*) = \alpha + \sum_{j = 1}^{m} \beta_j X_ij$$
3. Считаем тестовую статистику:

$$Z = \frac{n}{2} \left| ln(\frac{RSS_{Y}^*}{RSS_{log(Y)}^1}) \right|$$

Статистика Z имеет распределение $\chi^2$ с $1$-ой степенью свободы.

Для критерия $Z$ критическая область при уровне значимости  $\alpha$ - это область

$$\Omega_{\alpha}:\; Z > \chi^2_{1, \alpha}$$

где $\chi^2_{1, \alpha}$ - квантиль $\chi^2$-распределения.

Если гипотеза $H_0$ отвергается, то делается вывод о наличии значимой разницы в качестве оценивания и используется полулогарифмическая модель.

#### Пример теста Бокса-Кокса

$$TODO$$